<h1>Capstone 1: Data Wrangling (from API)</h1>

<a id='TOC'></a>
<strong>Table of Contents</strong>
<ol>
    <li>Preliminaries</li>
    <ol>
        <li><a href=#Sec01A>Import Modules</a></li>
    </ol>
    <li>Define Functions</li>
    <ol>
        <li><a href=#Sec02A>Remove Unnecessary Values</a></li>
        <li><a href=#Sec02B>Reduce Size of DataFrame</a></li>
        <li><a href=#Sec02C>Redesign the DataFrame</a></li>
    </ol>
    <li>Data Acquisition</li>
    <ol>
        <li><a href=#Sec03A>Inspect Sample of EMS Dataset</a></li>
        <li><a href=#Sec03B>Import Geographical Dataset</a></li>
    </ol>
    <li>Data Wrangling</li>
    <ol>
        <li><a href=#Sec04A>Apply Custom Functions</a></li>
        <li><a href=#Sec04B>Merge Clean Datasets</a></li>
        <li><a href=#Sec04C>Inspect Clean Dataset</a></li>
        <li><a href=#Sec04D>Export Clean Dataset</a></li>
        <li><a href=#Sec04E>Process Summary</a></li>
    </ol>
</ol>

The goal of this project is to develop machine learning models that predict whether or not the outcome of an EMS incident will result in a fatality. This is a supervised, binary classification problem. Analyses will be performed on a collection of nearly 8 million records of documented incidents, which span the six year period from January 2013 through December 2018, and appropriate predictive models will be developed to achieve the primary objective. This dataset is robust and contains several feature variables, of mixed data types, that describe both various attributes of each incident as well as the responsive action taken by the FDNY. All of the aforementioned factors affect an individual’s survivability once a response is initiated.

Data wrangling will be performed on two datasets for this analysis. One dataset contains EMS incident data spanning a six-year period. The second dataset contains geographical information for all ZIP Code Tabulation Areas within the City of New York.

<hr>

<h2 style="text-transform: uppercase;">1. Preliminaries</h2>

<a id='Sec01A'></a>
<h4>1A: Import Modules</h4>

In [1]:
# Import packages and modules
import pandas as pd
import numpy as np
import datetime

from sodapy import Socrata # library for Socrata Open Data API (SODA)

<p><a href=#TOC>TOC</a></p>

<hr>

<h2 style="text-transform: uppercase;">2. Define Functions</h2>

<a id='Sec02A'></a>
<h4>2A: Remove Unnecessary Values (drop_invalid and drop_immaterial)</h4>

For the purpose of this analysis, any observation with a missing value for `incident_disposition_code` must be omitted since the target variable is derived from this feature. In addition, observations that contain the following outliers, errors, or immaterial information must also be removed from the dataset:
<ul>
    <li>incidents created to transport a patient from one facility to another</li>
    <li>incidents where units were assigned to stand by in case they were needed</li>
    <li>incidents that pertain to special events</li>
    <li>incidents that were once closed but later reopened</li>
    <li>incidents with calculation errors for duration metrics</li>
    <li>features that contain redundant geographic information for incident</li>
</ul>

In [2]:
def drop_invalid(dfObj):
    # Drop all rows with missing value for select features
    dfObj.dropna(subset=['incident_disposition_code',
                         'zipcode',
                         'dispatch_response_seconds_qy',
                         'incident_travel_tm_seconds_qy',
                         'incident_response_seconds_qy'],
                 inplace=True)
    
    # Identify all columns that validate duration metrics
    list_of_validation_cols = [name for name in list(dfObj.columns)
                               if 'valid' in str(name)]
    
    # Drop all rows with invalid duration metrics
    for name in list_of_validation_cols:
        invalid_idx = dfObj[dfObj[name]=='N'].index
        dfObj.drop(invalid_idx, inplace=True)
    
    # Drop all rows where EMS were not dispatched
    no_disp_idx = dfObj[dfObj.dispatch_response_seconds_qy==0].index
    dfObj.drop(no_disp_idx, inplace=True)
    
    return dfObj

In [3]:
def drop_immaterial(dfObj):
    # Identify all columns with outlier event indicators
    list_of_indicator_cols = [name for name in list(dfObj.columns) 
                              if 'indicator' in str(name) and name !='held_indicator']
    
    # Drop all rows that pertain to outlier incidents
    for name in list_of_indicator_cols:
        outlier_idx = dfObj[dfObj[name]=='Y'].index
        dfObj.drop(outlier_idx, inplace=True)
    
    # Remove columns that contain incident indicator data
    dfObj.drop(list_of_indicator_cols,axis=1,inplace=True)
    dfObj.drop([name for name in list(dfObj.columns) 
                if '_indc' in str(name)],axis=1,inplace=True)
    
    # Identify and remove all columns that contain redundant geographic data
    list_of_zone_cols = [name for name in list(dfObj.columns)
                         if (('district' in str(name))| 
                             (name =='policeprecinct')| 
                             (name =='geoid'))]
    dfObj.drop(list_of_zone_cols,axis=1,inplace=True)
    
    return dfObj

<p><a href=#TOC>TOC</a></p>

<a id='Sec02B'></a>
<h4>2B: Reduce Size of DataFrame (reduce_memory)</h4>

<p>Modifying the data types for values contained within select columns will drastically reduce the memory usage of the dataframe object.</p>

In [4]:
def reduce_memory(dfObj):
    # Truncate name for borough label: 'RICHMOND / STATEN ISLAND'
    dfObj['borough'] = dfObj.borough.replace('RICHMOND / STATEN ISLAND',
                                             'STATEN ISLAND')
    
    # Create list of all columns that contain ISO8601 datetime
    list_of_datetime_cols = [name for name in list(dfObj.columns) 
                             if 'datetime' in str(name)]

    # Convert dtypes for each element in list to datetime
    for name in list_of_datetime_cols:
        dfObj[name] = pd.to_datetime(dfObj[name],errors='coerce')
       
    # Create list of all columns that contain time duration
    list_of_numeric_cols = [name for name in list(dfObj.columns) 
                            if (('seconds' in str(name))|
                                ('severity' in str(name))|
                                ('cad' in str(name)))]

    # Convert dtypes for each element in list to numeric
    for name in list_of_numeric_cols:
        dfObj[name] = pd.to_numeric(dfObj[name],errors='coerce')
    dfObj['incident_disposition_code'] = pd.to_numeric(dfObj.incident_disposition_code,errors='coerce')
        
    # Convert columns to category dtypes to reduce size of dataframe object
    dfObj['borough'] = dfObj.borough.astype('category')
    dfObj['zipcode'] = dfObj.zipcode.astype('category')
    dfObj['held_indicator'] = dfObj.held_indicator.astype('category')
    dfObj['incident_dispatch_area'] = dfObj.incident_dispatch_area.astype('category')
    
    return dfObj

<p><a href=#TOC>TOC</a></p>

<a id='Sec02C'></a>
<h4>2C: Redesign the DataFrame (format_df)</h4>

Construct a boolean series that represents the target variable (`fatality`) using the corresponding values in `incident_disposition_code`. Also, apply aesthetic changes to help improve the readability of the dataframe object.

In [5]:
def format_df(dfObj):
    # Rename select columns
    dfObj.rename(columns={'initial_severity_level_code':'initial_severity_level',
                          'final_severity_level_code':'final_severity_level',
                          'dispatch_response_seconds_qy':'dispatch_time',
                          'incident_travel_tm_seconds_qy':'travel_time',
                          'incident_response_seconds_qy':'response_time',
                          'intptlat':'latitude',
                          'intptlong':'longitude'},inplace=True)
    
    # Create a series for a new feature variable: life_threatening
    dfObj['life_threatening'] = [True if ((val ==1)|
                                         (val == 2)|
                                         (val == 3)) else False 
                                 for val in dfObj['final_severity_level'].astype('int64')]
    
    # Create a series for the target variable: fatality
    dfObj['fatality'] = np.logical_or(dfObj.incident_disposition_code.astype('int64') == 83,
                                      dfObj.incident_disposition_code.astype('int64') == 96)

    # Create separate columns for time components of the incident
    dfObj['year'] = pd.DatetimeIndex(dfObj.incident_datetime).strftime('%Y')
    dfObj['month'] = pd.DatetimeIndex(dfObj.incident_datetime).strftime('%m')
    dfObj['day'] = pd.DatetimeIndex(dfObj.incident_datetime).strftime('%d')
    dfObj['weekday'] = pd.DatetimeIndex(dfObj.incident_datetime).strftime('%w')
    dfObj['hour'] = pd.DatetimeIndex(dfObj.incident_datetime).strftime('%H')
    
    # Convert dtypes for each element in list to numeric
    list_of_time_cols = ['year','month','day','weekday','hour']
    for name in list_of_time_cols:
        dfObj[name] = pd.to_numeric(dfObj[name],errors='coerce')
    
    # Reorder columns of DataFrame object
    col_order = ['cad_incident_id','incident_datetime',
                 'year','month','day','hour','weekday','borough',
                 'zipcode','latitude','longitude','aland_sqmi','awater_sqmi',
                 'initial_call_type','initial_severity_level',
                 'final_call_type','final_severity_level',
                 'held_indicator','first_assignment_datetime',
                 'incident_dispatch_area',
                 'dispatch_time','first_activation_datetime',
                 'first_on_scene_datetime','travel_time','response_time',
                 'first_to_hosp_datetime','first_hosp_arrival_datetime',
                 'incident_close_datetime','incident_disposition_code',
                 'life_threatening','fatality']
    dfObj=dfObj[col_order]
    
    return dfObj

<p><a href=#TOC>TOC</a></p>

<hr>

<h2 style="text-transform: uppercase;">3. Data Acquisition</h2>

<a id='Sec03A'></a>
<h4>3A: Inspect Sample of EMS Incident Dataset</h4>

In [6]:
# Import sample of dataset via context manager
TIMEOUT = 60
with Socrata("data.cityofnewyork.us", None) as client:
    client.timeout = TIMEOUT
    results = client.get("66ae-7zpy", limit=1000)
    preview_df = pd.DataFrame.from_records(results)

In [7]:
preview_df.shape

(1000, 31)

In [8]:
preview_df.head()

,cad_incident_id,incident_datetime,initial_call_type,initial_severity_level_code,final_call_type,final_severity_level_code,first_assignment_datetime,valid_dispatch_rspns_time_indc,dispatch_response_seconds_qy,first_activation_datetime,...,zipcode,policeprecinct,citycouncildistrict,communitydistrict,communityschooldistrict,congressionaldistrict,reopen_indicator,special_event_indicator,standby_indicator,transfer_indicator
0,183654386,2018-12-31T23:59:46.000,CARDBR,2,CARDBR,2,2018-12-31T23:59:54.000,Y,8,2019-01-01T00:00:51.000,...,11201,84,35,302,13,8,N,N,N,N
1,183654385,2018-12-31T23:59:20.000,MVAINJ,4,MVAINJ,4,2019-01-01T00:01:10.000,Y,110,2019-01-01T00:02:25.000,...,11416,102,32,409,27,7,N,N,N,N
2,183654384,2018-12-31T23:58:45.000,SICK,6,SICK,6,2019-01-01T00:00:43.000,Y,118,2019-01-01T00:02:08.000,...,11369,115,21,480,30,14,N,N,N,N
3,183654383,2018-12-31T23:58:14.000,INJMAJ,3,INJMAJ,3,2018-12-31T23:58:22.000,Y,8,2018-12-31T23:58:37.000,...,11214,62,47,311,20,11,N,N,N,N
4,183654382,2018-12-31T23:57:42.000,UNC,2,UNC,2,2019-01-01T00:00:39.000,Y,177,2019-01-01T00:01:03.000,...,10019,18,4,105,2,12,N,N,N,N


In [9]:
preview_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 31 columns):
cad_incident_id                   1000 non-null object
incident_datetime                 1000 non-null object
initial_call_type                 1000 non-null object
initial_severity_level_code       1000 non-null object
final_call_type                   1000 non-null object
final_severity_level_code         1000 non-null object
first_assignment_datetime         992 non-null object
valid_dispatch_rspns_time_indc    1000 non-null object
dispatch_response_seconds_qy      1000 non-null object
first_activation_datetime         989 non-null object
first_on_scene_datetime           965 non-null object
valid_incident_rspns_time_indc    1000 non-null object
incident_response_seconds_qy      965 non-null object
incident_travel_tm_seconds_qy     965 non-null object
first_to_hosp_datetime            659 non-null object
first_hosp_arrival_datetime       650 non-null object
incident_close_dateti

<p><a href=#TOC>TOC</a></p>

<a id='Sec03B'></a>
<h4>3B: Import Geographical Dataset (from tab-delimited file)</h4>

In [10]:
# Import geographic data for all U.S. cities
"""
    The original TXT file can be exported from
    https://www2.census.gov/geo/docs/maps-data/data/gazetteer/2019_Gazetteer/2019_Gaz_zcta_national.zip
"""
input_path_geo = '../data/2019_Gaz_zcta_national.txt'

# Read tab-delimited text data into a Pandas DataFrame object
geo_df = pd.read_table(input_path_geo)

In [11]:
# Convert all column header names to lowercase
geo_df.rename(columns={i:i.lower().strip() for i in geo_df.columns},inplace=True)
geo_df.columns

Index(['geoid', 'aland', 'awater', 'aland_sqmi', 'awater_sqmi', 'intptlat',
       'intptlong'],
      dtype='object')

In [12]:
# Obtain shape of DataFrame w/ geographic data
geo_df.shape

(33144, 7)

<p>The text file contains GPA coordinates for all ZIP Code Tabulation Areas (ZCTAs) within the United States. A downloadable file is available at the <a href="https://www.census.gov/geographies/reference-files/time-series/geo/gazetteer-files.html" target="_blank">U.S. Census Bureau</a> under the file name <strong>ZIP Code Tabulation Areas</strong>. <a href=#TOC>TOC</a></p>

***

## 4. DATA WRANGLING

<a id='Sec04A'></a>
<h4>4A: Apply Custom Functions to Full EMS dataset (from NYC Open Data API)</h4>

In [13]:
frames = []
BATCH_SIZE = 500000

# Import dataset via context manager
with Socrata("data.cityofnewyork.us", None) as client:
    for i in range(18):
        
        # Get batch of EMS incidents dataset
        client.timeout = TIMEOUT
        results = client.get("66ae-7zpy", limit=BATCH_SIZE,offset=i*BATCH_SIZE)
        ems_df = pd.DataFrame.from_records(results)
        print(f'Batch {i+1} loaded to dataframe object...')
        
        # Join the EMS incidents and geographic datasets
        ems_df['zipcode'] = pd.to_numeric(ems_df['zipcode'],errors='coerce')
        temp_df = pd.merge(ems_df,geo_df,left_on='zipcode',right_on='geoid')
        
        # Apply all data cleansing functions
        temp_df = drop_invalid(temp_df)
        temp_df = drop_immaterial(temp_df)
        temp_df = reduce_memory(temp_df)
        temp_df = format_df(temp_df)
        print(f'Dataframe object (df{i+1}) cleaned')
        
        # Append cleaned temp_df to list
        frames.append(temp_df)
        print(f'Dataframe object (df{i+1}) appended to list')
        print()
print()        
print('All dataframe objects have been appended to list')

Batch 1 loaded to dataframe object...
Dataframe object (df1) cleaned
Dataframe object (df1) appended to list

Batch 2 loaded to dataframe object...
Dataframe object (df2) cleaned
Dataframe object (df2) appended to list

Batch 3 loaded to dataframe object...
Dataframe object (df3) cleaned
Dataframe object (df3) appended to list

Batch 4 loaded to dataframe object...
Dataframe object (df4) cleaned
Dataframe object (df4) appended to list

Batch 5 loaded to dataframe object...
Dataframe object (df5) cleaned
Dataframe object (df5) appended to list

Batch 6 loaded to dataframe object...
Dataframe object (df6) cleaned
Dataframe object (df6) appended to list

Batch 7 loaded to dataframe object...
Dataframe object (df7) cleaned
Dataframe object (df7) appended to list

Batch 8 loaded to dataframe object...
Dataframe object (df8) cleaned
Dataframe object (df8) appended to list

Batch 9 loaded to dataframe object...
Dataframe object (df9) cleaned
Dataframe object (df9) appended to list

Batch 10 l

<p>A downloadable description of each dataset field is available at <a href="https://data.cityofnewyork.us/Public-Safety/EMS-Incident-Dispatch-Data/76xm-jjuj" target="_blank">NYC Open Data</a> in the the <em>Attachments</em> section under the file name <strong>EMS_incident_dispatch_data_description.xlsx</strong>. <a href=#TOC>TOC</a></p>

<a id='Sec04B'></a>
<h4>4B: Merge Clean Datasets</h4>

In [14]:
# Concatenate all dataframe objects
df = pd.concat(frames,ignore_index=True)
print('Concatenated all dataframe objects in frames')

Concatenated all dataframe objects in frames


In [15]:
# Set index to 'cad_incident_id'
df.set_index(['cad_incident_id'],inplace=True)

<p><a href=#TOC>TOC</a></p>

<a id='Sec04C'></a>
<h4>4C: Inspect Clean Dataset</h4>

In [16]:
df.shape

(8054500, 30)

In [17]:
df.head()

,incident_datetime,year,month,day,hour,weekday,borough,zipcode,latitude,longitude,...,first_activation_datetime,first_on_scene_datetime,travel_time,response_time,first_to_hosp_datetime,first_hosp_arrival_datetime,incident_close_datetime,incident_disposition_code,life_threatening,fatality
cad_incident_id,,,,,,,,,,,,,,,,,,,,,
183654386,2018-12-31 23:59:46,2018,12,31,23,1,BROOKLYN,11201.0,40.6937,-73.989859,...,2019-01-01 00:00:51,2019-01-01 00:07:27,453,461,2019-01-01 00:15:08,2019-01-01 00:21:51,2019-01-01 01:00:20,82,True,False
183654358,2018-12-31 23:49:42,2018,12,31,23,1,BROOKLYN,11201.0,40.6937,-73.989859,...,2018-12-31 23:50:27,2018-12-31 23:55:10,303,328,2019-01-01 00:15:40,2019-01-01 00:28:05,2019-01-01 01:11:13,82,True,False
183654254,2018-12-31 23:24:58,2018,12,31,23,1,BROOKLYN,11201.0,40.6937,-73.989859,...,2018-12-31 23:25:37,2018-12-31 23:32:03,414,425,2019-01-01 00:03:55,2019-01-01 00:12:16,2019-01-01 00:45:31,82,False,False
183654178,2018-12-31 23:04:54,2018,12,31,23,1,BROOKLYN,11201.0,40.6937,-73.989859,...,2018-12-31 23:05:17,2018-12-31 23:16:13,668,679,NaT,NaT,2018-12-31 23:48:12,93,False,False
183653953,2018-12-31 22:08:50,2018,12,31,22,1,BROOKLYN,11201.0,40.6937,-73.989859,...,2018-12-31 22:09:35,2018-12-31 22:16:25,443,455,2018-12-31 22:27:08,2018-12-31 22:39:30,2018-12-31 23:05:01,82,True,False


In [18]:
df.info(verbose=True,null_counts=True,memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8054500 entries, 183654386 to 130091013
Data columns (total 30 columns):
incident_datetime              8054500 non-null datetime64[ns]
year                           8054500 non-null int64
month                          8054500 non-null int64
day                            8054500 non-null int64
hour                           8054500 non-null int64
weekday                        8054500 non-null int64
borough                        8054500 non-null object
zipcode                        8054500 non-null float64
latitude                       8054500 non-null float64
longitude                      8054500 non-null float64
aland_sqmi                     8054500 non-null float64
awater_sqmi                    8054500 non-null float64
initial_call_type              8054500 non-null object
initial_severity_level         8054500 non-null int64
final_call_type                8054500 non-null object
final_severity_level           8054500 non-nu

In [19]:
df.memory_usage(deep=True)

Index                           64436000
incident_datetime               64436000
year                            64436000
month                           64436000
day                             64436000
hour                            64436000
weekday                         64436000
borough                        537195915
zipcode                         64436000
latitude                        64436000
longitude                       64436000
aland_sqmi                      64436000
awater_sqmi                     64436000
initial_call_type              498729812
initial_severity_level          64436000
final_call_type                498825819
final_severity_level            64436000
held_indicator                   8054712
first_assignment_datetime       64436000
incident_dispatch_area         475215500
dispatch_time                   64436000
first_activation_datetime       64436000
first_on_scene_datetime         64436000
travel_time                     64436000
response_time   

<p><a href=#TOC>TOC</a></p>

<a id='Sec04D'></a>
<h4>4D: Export Clean Dataset (to CSV)</h4>

In [20]:
# Export dataframe to CSV
output_path = '../data/clean_EMS_data_from_api.csv'
print('Exporting dataframe to CSV...')
df.to_csv(output_path,index=False,compression='gzip')
print('Dataframe successfully exported to CSV using \'gzip\' compression.')

Exporting dataframe to CSV...
Dataframe successfully exported to CSV using 'gzip' compression.


<p><a href=#TOC>TOC</a></p>

<a id='Sec04E'></a>
<h4>4E: Process Summary</h4>

<p>The original EMS incident dataset was comprised of more than 8 million observations with 32 variables of mixed data types. The original geographic info dataset was comprised of 33,144 observations with 7 variables. The DataFrame objects generated from both source files were joined on the 'ZIP code' feature, though the labels had different names in each object.</p>

After all data pre-processing is complete, the resulting clean dataset consists of 7,988,028 observations of mixed data types, with a clear target variable and 30 feature variables. The target variable (`fatality`) was created by applying a boolean filter on the `incident_disposition_code` column within the DataFrame, which indicates the outcome of any EMS incident. Its output file occupies 342 MB of hard disk space and 3.7 GB in system memory. <p><a href=#TOC>TOC</a></p>